In [0]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (40.8.0)


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1013'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1013'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
!pip install tensorflow-gpu
!pip install tflearn
!pip install gym


In [0]:
# Set-up the virtual display environment 
!apt-get update
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet
!apt-get install ffmpeg -y

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (139 kB/s)
Reading package lists... Done
Reading package lists... Done
Building de

In [0]:
import gym
import random
import numpy as np
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import median, mean
from collections import Counter

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
from keras.models     import Sequential
from keras.layers     import Dense
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:


LR = 1e-3
#env = wrap_env(gym.make("Assault-v0"))
env = wrap_env(gym.make("CartPole-v1"))
#env = gym.make('CartPole-v1')

goal_steps = 500
score_requirement = 60
intial_games = 10000
             
observation = env.reset()


In [0]:
print(env.action_space)

Discrete(7)


In [0]:
def some_random_games_first():
    # Each of these is its own game.
    for episode in range(5):
        
        # this is each frame, up to 200...but we wont make it that far.
        for t in range(200):
            # This will display the environment
            # Only display if you really want to see it.
            # Takes much longer to display it.
            env.render()
            
            # This will just create a sample action in any environment.
            # In this environment, the action can be either of the 7 action spaces.
            action = env.action_space.sample()
            
            # this executes the environment with an action, 
            # and returns the observation of the environment, 
            # the reward, if the env is over, and other info.
            observation, reward, done, info = env.step(action)
            if done:
                break
                        
        env.close()
        show_video()
        
                
some_random_games_first()

In [0]:
def model_data_preparation():
    training_data = []
    accepted_scores = []
    for game_index in range(intial_games):
        score = 0
        game_memory = []
        previous_observation = []
        for step_index in range(goal_steps):
            action = random.randrange(0, 2)
            observation, reward, done, info = env.step(action)
            
            if len(previous_observation) > 0:
                game_memory.append([previous_observation, action])
                
            previous_observation = observation
            score += reward
            if done:
                break
            
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0, 1]
                elif data[1] == 0:
                    output = [1, 0]
                training_data.append([data[0], output])
        
        env.reset()

    print(accepted_scores)
    
    return training_data

In [0]:
def build_model(input_size, output_size):
    model = Sequential()
    model.add(Dense(128, input_dim=input_size, activation='relu'))
    model.add(Dense(52, activation='relu'))
    model.add(Dense(output_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam())
    return model


def train_model(training_data):
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]))
    y = np.array([i[1] for i in training_data]).reshape(-1, len(training_data[0][1]))
    
    model = build_model(input_size=len(X[0]), output_size=len(y[0]))
    
    model.fit(X, y, epochs=10)
    return model

In [0]:
training_data = model_data_preparation()

[64.0, 62.0, 62.0, 62.0, 67.0, 73.0, 63.0, 93.0, 98.0, 86.0, 93.0, 99.0, 61.0, 75.0, 68.0, 81.0, 69.0, 62.0, 69.0, 79.0, 63.0, 72.0, 64.0, 65.0, 62.0, 80.0, 77.0, 73.0, 76.0, 64.0, 64.0, 73.0, 66.0, 62.0, 76.0, 70.0, 67.0, 63.0, 66.0, 67.0, 83.0, 68.0, 74.0, 63.0, 74.0, 66.0, 73.0, 65.0, 71.0, 73.0, 82.0, 79.0, 63.0, 62.0, 60.0, 61.0, 85.0, 61.0, 62.0, 63.0, 62.0, 62.0, 69.0, 65.0, 91.0, 65.0, 92.0, 108.0, 64.0, 72.0, 71.0, 67.0, 66.0, 83.0, 66.0, 61.0, 73.0, 70.0, 60.0, 70.0, 66.0, 87.0, 64.0, 71.0, 73.0, 70.0, 61.0, 79.0, 80.0, 60.0, 63.0, 93.0, 74.0, 68.0, 84.0, 101.0, 67.0, 85.0, 85.0, 66.0, 70.0, 120.0, 70.0, 69.0, 71.0, 76.0, 70.0, 74.0, 70.0, 68.0, 68.0, 77.0, 62.0, 73.0, 61.0, 68.0, 60.0, 60.0, 62.0, 80.0, 75.0, 67.0, 61.0, 72.0, 68.0, 91.0, 87.0, 61.0, 61.0, 67.0, 80.0, 97.0, 60.0, 67.0, 75.0, 61.0, 78.0, 86.0, 88.0, 65.0]


In [0]:
trained_model = train_model(training_data)

Epoch 1/10
9934/9934 [==============================] - 1s 143us/step - loss: 0.2491
Epoch 2/10
9934/9934 [==============================] - 1s 120us/step - loss: 0.2345
Epoch 3/10
9934/9934 [==============================] - 1s 123us/step - loss: 0.2337
Epoch 4/10
9934/9934 [==============================] - 1s 120us/step - loss: 0.2329
Epoch 5/10
9934/9934 [==============================] - 1s 120us/step - loss: 0.2326
Epoch 6/10
9934/9934 [==============================] - 1s 120us/step - loss: 0.2320
Epoch 7/10
9934/9934 [==============================] - 1s 120us/step - loss: 0.2321
Epoch 8/10
9934/9934 [==============================] - 1s 120us/step - loss: 0.2310
Epoch 9/10
9934/9934 [==============================] - 1s 120us/step - loss: 0.2312
Epoch 10/10
9934/9934 [==============================] - 1s 121us/step - loss: 0.2315


In [0]:
scores = []
choices = []
for each_game in range(100):
    score = 0
    prev_obs = []
    for step_index in range(goal_steps):
        env.render()
        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(trained_model.predict(prev_obs.reshape(-1, len(prev_obs)))[0])
        
        choices.append(action)
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        score+=reward
        if done:
            break
    env.close()
    show_video()
    env.reset()
    scores.append(score)
    
print(scores)
print('Average Score:', sum(scores)/len(scores))
print('choice 1:{}  choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))


[287.0, 172.0, 173.0, 334.0, 340.0, 500.0, 500.0, 184.0, 500.0, 299.0, 500.0, 496.0, 408.0, 295.0, 500.0, 317.0, 153.0, 500.0, 168.0, 292.0, 500.0, 500.0, 211.0, 500.0, 276.0, 300.0, 307.0, 359.0, 275.0, 500.0, 154.0, 169.0, 125.0, 145.0, 269.0, 321.0, 175.0, 309.0, 500.0, 500.0, 277.0, 163.0, 500.0, 147.0, 148.0, 500.0, 129.0, 457.0, 162.0, 362.0, 144.0, 308.0, 500.0, 147.0, 394.0, 305.0, 156.0, 500.0, 476.0, 243.0, 500.0, 161.0, 171.0, 355.0, 500.0, 143.0, 341.0, 500.0, 345.0, 353.0, 500.0, 500.0, 325.0, 159.0, 500.0, 500.0, 252.0, 143.0, 500.0, 276.0, 500.0, 500.0, 388.0, 147.0, 157.0, 187.0, 175.0, 361.0, 294.0, 500.0, 155.0, 267.0, 161.0, 298.0, 500.0, 500.0, 500.0, 500.0, 162.0, 285.0]
Average Score: 328.92
choice 1:0.49054481332846894  choice 0:0.5094551866715311
